In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [3]:

import json
import re

def process_json_text2(text=None):
    """
    Process a JSON file or string and extract the 'global_argument' and 'comments' fields.
    Handles JSON strings that might be wrapped in code blocks or have escape characters.
    
    Args:
        file_path (str, optional): Path to the JSON file
        json_str (str, optional): JSON string to process
    
    Returns:
        tuple: (global_argument, comments)
    """
 
    # Clean the input text
    # Remove code block markers if present
    text = re.sub(r'^```json\s*|\s*```$', '', text, flags=re.MULTILINE)
    
    # Try to parse the JSON
    try:
        data = json.loads(text)
    except json.JSONDecodeError:
        # If direct parsing fails, try to extract the JSON portion
        # Look for patterns that might indicate JSON content
        # This pattern handles cases with or without comments field
        json_pattern = r'{\s*"global_argument"\s*:.*?(?:"comments"\s*:.*?)?}'
        match = re.search(json_pattern, text, re.DOTALL)
        if match:
            try:
                # Try to parse the extracted JSON
                extracted_json = match.group(0)
                data = json.loads(extracted_json)
            except json.JSONDecodeError:
                # If this still fails, we might need to handle escaped characters or missing commas
                clean_text = match.group(0).replace('\\"', '"').replace('\\n', '\n')
                # Remove any remaining escapes that might interfere
                clean_text = re.sub(r'\\(.)', r'\1', clean_text)
                
                # Fix missing comma between global_argument and comments
                # Look for pattern: "text" "comments" and insert comma between them
                clean_text = re.sub(r'("\s*)\s*("comments"\s*:)', r'\1, \2', clean_text)
                
                try:
                    data = json.loads(clean_text)
                except json.JSONDecodeError:
                    # Last attempt: manually extract the values using regex
                    global_arg_match = re.search(r'"global_argument"\s*:\s*"([^"]*(?:\\.[^"]*)*)"', clean_text, re.DOTALL)
                    comments_match = re.search(r'"comments"\s*:\s*"([^"]*(?:\\.[^"]*)*)"', clean_text, re.DOTALL)
                    
                    if global_arg_match:
                        global_argument = global_arg_match.group(1).replace('\\"', '"').replace('\\n', '\n')
                        comments = comments_match.group(1).replace('\\"', '"').replace('\\n', '\n') if comments_match else ""
                        return global_argument, comments
                    else:
                        return "Could not extract global_argument from malformed JSON 1" , 'TEXT'
        else:
            return "Could not extract valid JSON from the input 2" , 'NO TEXT'
    
    # Extract the required fields
    global_argument = data.get("global_argument", "")
    comments = data.get("comments", "")
    
    return global_argument, comments

def process_json_text(text=None):
    """
    Process a JSON file or string and extract the 'global_argument' and 'comments' fields.
    Handles JSON strings that might be wrapped in code blocks or have escape characters.
    
    Args:
        file_path (str, optional): Path to the JSON file
        json_str (str, optional): JSON string to process
    
    Returns:
        tuple: (global_argument, comments)
    """
    
    # Clean the input text
    # Remove code block markers if present
    text = re.sub(r'^```json\s*|\s*```$', '', text, flags=re.MULTILINE)
    # Fix trailing commas in objects
    fixed = re.sub(r',\s*}', r'}', text)
    # Fix trailing commas in arrays
    text = re.sub(r',\s*]', r']', fixed)

    # Try to parse the JSON
    try:
        data = json.loads(text)
    except json.JSONDecodeError:
        # If direct parsing fails, try to extract the JSON portion
        # Look for patterns that might indicate JSON content
       # json_pattern = r'{\s*"global_argument"\s*:.*?"comments"\s*:.*?}'
        json_pattern = r'{\s*"global_argument"\s*:.*?(?:"comments"\s*:.*?)?}'
        match = re.search(json_pattern, text, re.DOTALL)
        if match:
            try:
                # Try to parse the extracted JSON
                data = json.loads(match.group(0))
            except json.JSONDecodeError:
                # If this still fails, we might need to handle escaped characters
                clean_text = match.group(0).replace('\\"', '"').replace('\\n', '\n')
                # Remove any remaining escapes that might interfere
                clean_text = re.sub(r'\\(.)', r'\1', clean_text)
                data = json.loads(clean_text)
        else:
            return "Could not extract valid JSON from the input for this: " + text, 'NO JSON'
    
    # Extract the required fields
    global_argument = data.get("global_argument", "")
    comments = data.get("comments", "")
    
    return global_argument, comments

def process_json(row=None):
    return process_json_text2(row.response)

def remove_think(row):
    """
    Extracts all content between <think> and </think> tags and returns both
    the extracted content and the remaining text.
    
    Args:
        text (str): The input text string
        
    Returns:
        tuple: (think_text, remaining_text)
            think_text (str): The extracted <think>...</think> content including tags
            remaining_text (str): The text with all <think>...</think> sections removed
    """
    text = row.response
    # Pattern to match <think> tag and its content until </think>
    pattern = r'<think>.*?</think>'
    
    # Find all matches
    matches = re.findall(pattern, text, flags=re.DOTALL)
    
    # Combine all matched <think> sections
    think_text = ''.join(matches) if matches else ''
    
    # Remove matching patterns with re.sub
    remaining_text = re.sub(pattern, '', text, flags=re.DOTALL)
    
    # Strip any extra whitespace that might remain
    remaining_text = remaining_text.strip()
    
    return think_text, remaining_text


In [9]:
df_stance = pd.read_csv('7-topic_color_stance_repres-doc.csv')

In [10]:
df_stance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  2971 non-null   int64  
 1   Unnamed: 0    2971 non-null   int64  
 2   0             2971 non-null   object 
 3   1             2971 non-null   object 
 4   2             2971 non-null   object 
 5   3             2971 non-null   object 
 6   4             2971 non-null   object 
 7   5             2971 non-null   object 
 8   6             2971 non-null   object 
 9   color         2971 non-null   float64
 10  stance        2971 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 255.4+ KB


## LLM Gemma


In [ ]:
df_gemma = pd.read_csv('20250811-111040gemma_1shot_all_v1.csv')

df_gemma['color'] = df_stance['color']
df_gemma['stance'] = df_stance['stance']

res = df_gemma.apply(process_json, axis=1)
df_gemma['global_argument'], df_gemma['comments'] = zip(*res)




In [37]:
df_gemma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   topic            2971 non-null   int64  
 1   response         2971 non-null   object 
 2   prompt           2971 non-null   object 
 3   timestamp        2971 non-null   object 
 4   op1              2971 non-null   object 
 5   op2              2971 non-null   object 
 6   op3              2971 non-null   object 
 7   duration         2971 non-null   float64
 8   color            2971 non-null   float64
 9   stance           2971 non-null   object 
 10  global_argument  2971 non-null   object 
 11  comments         2971 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 278.7+ KB


In [38]:
df_gemma['stance'].value_counts()

stance
favor        2787
against       132
polarized      52
Name: count, dtype: int64

In [39]:
df_gemma.to_csv('gemma_all_args.csv',index=False)

In [17]:
df_gemma

,topic,response,prompt,timestamp,op1,op2,op3,duration,global_argument,comments,color,stance
0,0,"```json\n{\n ""global_argument"": ""It is observed that a significant number of candidates are being positioned as committed to addressing the issue of climate change, and a considerable effort is being directed towards framing this as a central priority. The idea of candidates being targeted for their stances on this issue is being actively promoted, and a considerable amount of support is being mobilized around the concept of prioritizing climate action among the broader field of political contenders. It is recognized that the momentum surrounding this focus is being leveraged to influence the debate and shape the policy landscape."",\n ""comments"": ""The argument synthesizes the opinions by highlighting the active promotion of Jay Inslee's climate change stance and the resulting debate surrounding it. It emphasizes the movement being built around this issue and its potential to influence the broader political landscape, all expressed in passive voice.""\n}\n```","Your task is to construct a compelling global argument supporting the widely held belief that human activity is the dominant driver of current climate change. You are not to explicitly state that humans cause climate change, but rather to present a perspective that recognizes the extent of human influence. To do this, don't add new information; only use the information provided by the opinions given.\nTask: Constructing a Global Short Argument in Passive Voice Challenging that Climate Change is Caused by Humans. \nObjective: To synthesize multiple opinions into a single, coherent global short argument that reflects the shared concerns or themes, using passive voice and using only opinions provided without adding any new information.\nInstructions:\nYou are given a list of opinions that address a common issue or theme, possibly from different perspectives. Your task is to:\n1. Identify the main concerns expressed in the opinions.\n2. Integrate these concerns into a single, unified global argument.\n3. Write the global argument entirely in passive voice, presenting a cohesive perspective that reflects the central tension or goal shared among the individual opinions. Using only opinnions provided without adding any new information.\n4. Return your output in JSON format using the following structure:\n{\n""global_argument"": ""[Your global argument written in passive voice]"",\n""comments"":""[if you want to add comments or notes in reference to the output]""\n}\nThis is an example\n**Example Input:**\n{\n ""opinions"": [\n ""Rising global temperatures are causing more frequent and severe wildfires."",\n ""Deforestation contributes to climate change by reducing the number of trees that absorb CO2."",\n ""Governments should invest in reforestation programs to combat climate change.""\n ]\n}\n**Output for the Example Input:**\n{\n ""global_argument"": ""It is widely acknowledged that rising global temperatures are causing wildfires to become more frequent and severe, with the number of trees that absorb CO2 being significantly reduced due to deforestation. As a result, climate change is being exacerbated, and the restoration of carbon-absorbing trees through reforestation programs is seen as a necessary solution. It is urged that investments in such programs be made by governments to help mitigate the ongoing effects of climate change."",\n ""comments"": ""The argument connects with the opinions by highlighting how climate change drives wildfires and deforestation, thereby reinforcing the need for reforestation programs to restore trees that absorb CO2 and combat further climate change.""\n}\nEND OF EXAMPLE.\n-------------\nNow, construct a global argument for this new input\n**Input:**\n{""opinions"": [""@mediaknowitall Jay Inslee is the candidate committed to combatting climate change. This is our moment! Join the movement: https://t.co/Dw7y86VVWN https://t.co/ZWh4frZnhE "

In [31]:
df_filtrado = df_gemma[df_gemma['stance'] == 'polarized']


In [32]:
df_filtrado

,topic,response,prompt,timestamp,op1,op2,op3,duration,color,stance,global_argument,comments
11,11,"```json\n{\n ""global_argument"": ""It has been asserted that there is no scientific proof of anthropogenic climate change, and that the assertion of ‘settled science’ has been used to ignore contrary evidence; it has been suggested that government grants corrupt science, and that the focus on climate change has been driven by specialized research, assuming that there’s time for its results to be beneficial."",\n ""comments"": ""The synthesized argument reflects the core skepticism expressed across the provided opinions, emphasizing the lack of definitive proof, the potential for bias in scientific research, and the passive framing of the issue as a matter of specialized investigation rather than a demonstrable human cause.""\n}\n```","Your task is to construct a compelling global argument synthesizing the opinions regarding the belief that human activity is the dominant driver of current climate change. To do this, don't add new information; only use the information provided by the opinions given.\nTask: Synthesize a Global Short Argument in Passive Voice Challenging that Climate Change is Caused by Humans. \nObjective: To synthesize multiple opinions into a single, coherent global short argument that reflects the shared concerns or themes, using passive voice and using only opinions provided without adding any new information.\nInstructions:\nYou are given a list of opinions that address a common issue or theme, possibly from different perspectives. Your task is to:\n1. Identify the main concerns expressed in the opinions.\n2. Integrate these concerns into a single, unified global argument.\n3. Write the global argument entirely in passive voice, presenting a cohesive perspective that reflects the central tension or goal shared among the individual opinions. Using only opinions provided without adding any new information.\n4. Return your output in JSON format using the following structure:\n{\n""global_argument"": ""[Your global argument written in passive voice]"",\n""comments"":""[if you want to add comments or notes in reference to the output]""\n}\nThis is an example\n**Example Input:**\n{\n ""opinions"": [\n ""Rising global temperatures are causing more frequent and severe wildfires."",\n ""Deforestation contributes to climate change by reducing the number of trees that absorb CO2."",\n ""Current climate changes have been attributed primarily to natural cycles and variations in solar activity rather than to human influence.""\n ]\n}\n**Output for the Example Input:**\n{\n ""global_argument"": ""Rising global temperatures have been linked to more frequent and severe wildfires, while deforestation has been recognized as reducing the capacity to absorb CO₂; however, current climate changes have also been attributed primarily to natural cycles and variations in solar activity rather than to human influence."",\n ""comments"": ""The synthesized version takes the three opinions and merges them into one coherent statement that keeps their meaning intact, uses passive voice, and preserves both supportive and opposing perspectives.""\n}\nEND OF EXAMPLE.\n-------------\nNow, construct a global argument for this new input\n**Input:**\n{""opinions"": [""@AdamRamsay The real answer to the Guardian question is none! There is no scientific proof of anthropogenic climate change, if you know of some enlighten us all!🙄 "",\n""@multtskr2 @kindamuslim @Parkerfabulous @Jordan_Sather_ “The science is settled” always makes me laugh, whether it’s used with global warming, evolution or vaccines. Yeah, it’s settled as long as you ignore all evidence to the contrary. Government grants corrupt science, like everything else. "",\n""Scientists specialise. So, while climate scientists are exhorting us to take rapid action to counter global warming due to mankind’s accelerating use of fossil fuels, the rest are beavering way on important research, assumi

## Llama

In [40]:
df_llama = pd.read_csv('20250812-112614llama_1shot_all_v1.csv')

df_llama['color'] = df_stance['color']
df_llama['stance'] = df_stance['stance']

res = df_llama.apply(process_json, axis=1)
df_llama['global_argument'], df_llama['comments'] = zip(*res)




In [42]:
df_llama.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   topic            2971 non-null   int64  
 1   response         2971 non-null   object 
 2   prompt           2971 non-null   object 
 3   timestamp        2971 non-null   object 
 4   op1              2971 non-null   object 
 5   op2              2971 non-null   object 
 6   op3              2971 non-null   object 
 7   duration         2971 non-null   float64
 8   color            2971 non-null   float64
 9   stance           2971 non-null   object 
 10  global_argument  2971 non-null   object 
 11  comments         2971 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 278.7+ KB


## Deepseek

In [41]:
df_ds = pd.read_csv('20250813-001452deepseek_1shot_all_v1.csv')

df_ds['color'] = df_stance['color']
df_ds['stance'] = df_stance['stance']

res = df_ds.apply(process_json, axis=1)
df_ds['global_argument'], df_ds['comments'] = zip(*res)


In [43]:
df_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   topic            2971 non-null   int64  
 1   response         2971 non-null   object 
 2   prompt           2971 non-null   object 
 3   timestamp        2971 non-null   object 
 4   op1              2971 non-null   object 
 5   op2              2971 non-null   object 
 6   op3              2971 non-null   object 
 7   duration         2971 non-null   float64
 8   color            2971 non-null   float64
 9   stance           2971 non-null   object 
 10  global_argument  2971 non-null   object 
 11  comments         2971 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 278.7+ KB


### Create master arguments dataframe 

In [49]:
data = {
    'topic': df_gemma['topic'].to_list(),
    'op1': df_gemma['op1'].to_list(),
    'op2': df_gemma['op2'].to_list(),
    'op3': df_gemma['op3'].to_list(),
    'argument_gemma': df_gemma['global_argument'].to_list(),
    'argument_llama': df_llama['global_argument'].to_list(),
    'argument_ds': df_ds['global_argument'].to_list(),
    
}
df_marter_args = pd.DataFrame(data)

In [50]:
df_marter_args.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   topic           2971 non-null   int64 
 1   op1             2971 non-null   object
 2   op2             2971 non-null   object
 3   op3             2971 non-null   object
 4   argument_gemma  2971 non-null   object
 5   argument_llama  2971 non-null   object
 6   argument_ds     2971 non-null   object
dtypes: int64(1), object(6)
memory usage: 162.6+ KB


In [53]:
count = df_marter_args['argument_gemma'].str.startswith('Could not extract valid', na=False).sum()
print(f"Count: {count}")

Count: 0


In [54]:
count = df_marter_args['argument_llama'].str.startswith('Could not extract valid', na=False).sum()
print(f"Count: {count}")

Count: 302


In [55]:
count = df_marter_args['argument_ds'].str.startswith('Could not extract valid', na=False).sum()
print(f"Count: {count}")

Count: 20


In [52]:
df_marter_args.head(30)

,topic,op1,op2,op3,argument_gemma,argument_llama,argument_ds
0,0,@mediaknowitall Jay Inslee is the candidate committed to combatting climate change. This is our moment! Join the movement: https://t.co/Dw7y86VVWN https://t.co/ZWh4frZnhE,@AustinJenkinsN3 I'm sure all the other candidates will look at this attack and then think what a great idea it would be to have all the candidates attacked by Inslee in a 'climate change' debate,Glad to see @BetoORourke join @JayInslee as real leaders on #climatechange — looking forward to seeing this become the #1 priority among the rest of the candidates. https://t.co/p43pIvIzyg,"It is observed that a significant number of candidates are being positioned as committed to combating climate change, and a considerable effort is being directed towards framing this issue as a central priority. The idea of candidates being targeted for their stances on this matter is being actively promoted, suggesting a widespread recognition of the importance of addressing climate change. Furthermore, the focus on this issue is being amplified, indicating a collective desire to see it become a primary concern among the broader field of political contenders.","It is widely acknowledged that climate change has become a pressing issue, with various political figures recognizing its importance and committing to combat it. The increasing attention on this topic is seen as a positive development, as it brings together individuals who share similar concerns about the future of our planet. It is urged that these leaders continue to prioritize climate change as a key aspect of their platforms, in order to inspire a broader movement towards sustainability and environmental protection.","It is widely recognized that there are numerous individuals committed to combating climate change, with Jay Inslee and Beto O'Rourke being among them. These real leaders have been advocating for the issue to become a top priority among other candidates in the political sphere. As such, it is urged that more support be given to these advocates of climate action, as they are seen as key figures in driving meaningful change on this critical global challenge."
1,1,"Time running out 2 stop catastrophic #climatechange. leaders have let us down - but people worldwide are taking action, from community energy to #schoolstrikes. Join me &amp; @friends_earth to demand Government urgent #climateaction. https://t.co/EwXqk8ZPQa","ccchangemakers: Time is running out to stop catastrophic #climatechange. Our leaders have let us down - but people worldwide are taking action, from community energy to #schoolstrikes. Join me &amp; friends_earth to demand that the UK government takes urgent… https://t.co/I1tTeXEEe8","Thank God for the young people who can see past #Brexit to the real crisis that is facing us #climatechange. Our leaders have let us down - but people taking action, from community energy to #schoolstrikes. Demand UK Gvt. takes urgent #climateaction. https://t.co/a4q9Lml8Mc","It is increasingly recognized that significant challenges are being presented by the escalating impacts of climate change globally. The urgency of the situation is being underscored by widespread concern, with action being taken by communities and individuals worldwide, including through initiatives like community energy and school strikes. The responsibility for addressing this crisis is being increasingly attributed to those in positions of leadership, who are perceived to have failed to adequately respond. Consequently, demands for immediate and decisive action are being made, emphasizing the need for governments to prioritize and implement effective solutions to mitigate the ongoing effects of climate change.","It is widely acknowledged that time is running out to stop catastrophic climate change, and the lack of urgency from leaders has been disappointing. However, people worldwide are taking action, from community energy initiatives to school strikes, which is seen as a p

In [56]:
df_marter_args.to_csv('master_args.csv')